In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("input/merged_mArray_RNAseq.csv") #

In [3]:
df.columns

Index(['Unnamed: 0', 'mArray_stat', 'mArray_padj', 'mArray_log2FC_TCvsB6',
       'RNAseq_baseMean', 'RNAseq_log2FoldChange', 'RNAseq_lfcSE',
       'RNAseq_stat', 'RNAseq_pvalue', 'RNAseq_padj'],
      dtype='object')

#### Check column names before moving on
------------

In [4]:
gene_list_df = pd.read_csv("input/Tfh_specific_genes_Immunity2014_v2.csv") #

In [5]:
gene_l = gene_list_df['X'].tolist()

In [6]:
new_l = []
for item in df['Unnamed: 0']:
    if item in gene_l:
        new_l.append(item)
    else:
        new_l.append("NA")

In [7]:
df['annot'] = new_l

In [8]:
df.to_csv("output/merged_mArray_RNAseq_annot.csv",index=False) #

-------------

# anootate pathway
* mitochondria

In [9]:
import pandas as pd

In [10]:
oxphos_gl = pd.read_csv("input/gl_KEGG_OXIDATIVE_PHOSPHORYLATION_4HTprep.csv")
TCA_gl = pd.read_csv("input/gl_KEGG_CITRATE_CYCLE_TCA_CYCLE_4HTprep.csv")
ribosome_gl = pd.read_csv("input/gl_KEGG_RIBOSOME_HT4R.csv")

In [11]:
oxphos_gl['metabolism'] = 'OXPHOS'
TCA_gl['metabolism'] = 'TCA_cycle'
ribosome_gl['metabolism'] = 'Ribosome'

In [12]:
gl_df = pd.concat([oxphos_gl,TCA_gl,ribosome_gl],axis =0)

In [13]:
gl_df.shape

(205, 2)

In [14]:
a_list = gl_df['mouseSymbol'].tolist()

In [15]:
duplicate_gl = []
for item in a_list:
    if a_list.count(item) > 1:
        duplicate_gl.append(item)

In [16]:
uni_gl = [x for x in set(gl_df['mouseSymbol']) if x not in duplicate_gl]

In [17]:
uni_df = gl_df.loc[gl_df['mouseSymbol'].isin(uni_gl),:]

In [18]:
duplicate_gl = list(set(duplicate_gl))

In [19]:
dup_df_cor = pd.DataFrame({"mouseSymbol": duplicate_gl, "metabolism": ["OXPHOS"]*len(duplicate_gl)})

In [20]:
df_met_annot = pd.concat([uni_df,dup_df_cor],axis=0)

In [21]:
len(set(df_met_annot['mouseSymbol']))

201

In [22]:
df.columns

Index(['Unnamed: 0', 'mArray_stat', 'mArray_padj', 'mArray_log2FC_TCvsB6',
       'RNAseq_baseMean', 'RNAseq_log2FoldChange', 'RNAseq_lfcSE',
       'RNAseq_stat', 'RNAseq_pvalue', 'RNAseq_padj', 'annot'],
      dtype='object')

In [23]:
df2 = pd.merge(df,df_met_annot,left_on = "Unnamed: 0",right_on = "mouseSymbol",how = "left")

# annotate statistics
* co-up general 
* co-up FC1.5
* co-down general
* co-down FC1.5

In [24]:
df2.columns

Index(['Unnamed: 0', 'mArray_stat', 'mArray_padj', 'mArray_log2FC_TCvsB6',
       'RNAseq_baseMean', 'RNAseq_log2FoldChange', 'RNAseq_lfcSE',
       'RNAseq_stat', 'RNAseq_pvalue', 'RNAseq_padj', 'annot', 'mouseSymbol',
       'metabolism'],
      dtype='object')

In [25]:
df2['stat_annot'] = np.nan

In [26]:
df2.loc[(df2['mArray_padj'] < 0.05)
        & (df2['RNAseq_padj'] < 0.05)
        & (df2['RNAseq_log2FoldChange'] > 0) 
        & (df2['mArray_log2FC_TCvsB6'] > 0),"stat_annot"] = "co_high_FC0"

In [27]:
df2.loc[(df2['mArray_padj'] < 0.05)
        & (df2['RNAseq_padj'] < 0.05)
        & (df2['RNAseq_log2FoldChange'] < 0) 
        & (df2['mArray_log2FC_TCvsB6'] < 0),"stat_annot"] = "co_low_FC0"

In [28]:
df2.loc[(df2['mArray_padj'] < 0.05)
        & (df2['RNAseq_padj'] < 0.05)
        & (df2['RNAseq_log2FoldChange'] > 0.5850) 
        & (df2['mArray_log2FC_TCvsB6'] > 0.5850),"stat_annot"] = "co_high_FC1.5"

In [29]:
df2.loc[(df2['mArray_padj'] < 0.05)
        & (df2['RNAseq_padj'] < 0.05)
        & (df2['RNAseq_log2FoldChange'] < -0.5850) 
        & (df2['mArray_log2FC_TCvsB6'] < -0.5850),"stat_annot"] = "co_low_FC1.5"

In [30]:
df2.to_csv("output/merged_mArray_RNAseq_annot_meta_stat_v2.csv",index=False)

In [31]:
df2.loc[(df2['RNAseq_log2FoldChange'] >5) ,'RNAseq_log2FoldChange'] = 5
df2.loc[(df2['RNAseq_log2FoldChange'] <-5) ,'RNAseq_log2FoldChange'] = -5

In [32]:
df2.loc[(df2['mArray_log2FC_TCvsB6'] >5) ,'mArray_log2FC_TCvsB6'] = 5
df2.loc[(df2['mArray_log2FC_TCvsB6'] <-5) ,'mArray_log2FC_TCvsB6'] = -5

In [33]:
df2.to_csv("output/merged_mArray_RNAseq_annot_meta_stat_log2FC_max5_v3.csv",index=False)